# Automatic differentiation with torch.autograd
When training neural networks, the most frequently used algorithm is back propagation. In this algorithm, parameters (model weights) are adjusted according to the gradient of the loss function with respect to the given parameter.

To compute those gradients, PyTorch has a built-in differentiation engine called torch.autograd. It supports automatic computation of gradient for any computational graph.

Consider the simplest one-layer neural network, with input x, parameters w and b, and some loss function. It can be defined in PyTorch in the following manner:

In [1]:
import torch

x=torch.ones(5)  # input tensor
y=torch.zeros(3) # expected output
w=torch.rand(5,3,requires_grad=True)
b=torch.rand(3,requires_grad=True)
z=torch.matmul(x,w)+b
loss=torch.nn.functional.binary_cross_entropy_with_logits(z,y)

## Tensors,Functions and Computational graph

In [2]:
print(f"Gradient function for z={z.grad_fn}")
print(f"Gradient function for loss={loss.grad_fn}")

Gradient function for z=<AddBackward0 object at 0x7c8ff4e45720>
Gradient function for loss=<BinaryCrossEntropyWithLogitsBackward0 object at 0x7c8ff4e451e0>


## Computing Gradients

In [3]:
loss.backward()
print(w.grad)
print(b.grad)

tensor([[0.3221, 0.2995, 0.3149],
        [0.3221, 0.2995, 0.3149],
        [0.3221, 0.2995, 0.3149],
        [0.3221, 0.2995, 0.3149],
        [0.3221, 0.2995, 0.3149]])
tensor([0.3221, 0.2995, 0.3149])


## Disabling Gradient Tracking
By default, all tensors with `requires_grad=True` are tracking their computational history and support gradient computation. However, there are some cases when we do not need to do that, for example, when we have trained the model and just want to apply it to some input data, i.e. we only want to do forward computations through the network. We can stop tracking computations by surrounding our computation code with `torch.no_grad()` block:

In [4]:
z=torch.matmul(x,w)+b
print(z.requires_grad)

with torch.no_grad():
  z=torch.matmul(x,w)+b
print(z.requires_grad)

True
False


In [5]:
z=torch.matmul(x,w)+b
z_det=z.detach()
print(z_det.requires_grad)

False


There are reasons you might want to disable gradient tracking:
*   To mark some parameters in your neural network as frozen parameters.

*   To speed up computations when you are only doing forward pass, because computations on tensors that do not track gradients would be more efficient.


## More on Computational Graphs
Conceptually, autograd keeps a record of data (tensors) and all executed operations (along with the resulting new tensors) in a directed acyclic graph (DAG) consisting of Function objects. In this DAG, leaves are the input tensors, roots are the output tensors. By tracing this graph from roots to leaves, you can automatically compute the gradients using the chain rule.

In a forward pass, autograd does two things simultaneously:


*   run the requested operation to compute a resulting tensor
*   maintain the operation’s gradient function in the DAG.

The backward pass kicks off when `.backward()` is called on the DAG root. `autograd` then:

*   computes the gradients from each `.grad_fn`,
*   accumulates them in the respective tensor’s .grad attribute
*   using the chain rule, propagates all the way to the leaf tensors.





## Optional Reading: Tensor Gradients and Jacobian Products

In [6]:
inp = torch.eye(4,5,requires_grad=True)
out = (inp+1).pow(2).t()
out.backward(torch.ones_like(out),retain_graph=True)
print(f"First call\n{inp.grad}")
out.backward(torch.ones_like(out), retain_graph=True)
print(f"\nSecond call\n{inp.grad}")
inp.grad.zero_()
out.backward(torch.ones_like(out), retain_graph=True)
print(f"\nCall after zeroing gradients\n{inp.grad}")

First call
tensor([[4., 2., 2., 2., 2.],
        [2., 4., 2., 2., 2.],
        [2., 2., 4., 2., 2.],
        [2., 2., 2., 4., 2.]])

Second call
tensor([[8., 4., 4., 4., 4.],
        [4., 8., 4., 4., 4.],
        [4., 4., 8., 4., 4.],
        [4., 4., 4., 8., 4.]])

Call after zeroing gradients
tensor([[4., 2., 2., 2., 2.],
        [2., 4., 2., 2., 2.],
        [2., 2., 4., 2., 2.],
        [2., 2., 2., 4., 2.]])
